# Adaptive

[`adaptive`](https://gitlab.kwant-project.org/basnijholt/adaptive-evaluation) is a package for adaptively sampling functions with support for parallel evaluation.

This is an introductory notebook that shows some basic use cases.

`adaptive` needs the following packages:

+ Python 3.6
+ holowiews
+ bokeh

In [ ]:
import adaptive
adaptive.notebook_extension()

# 1D function learner

We start with the most common use-case: sampling a 1D function $\ f: ℝ → ℝ$.

We will use the following function, which is a smooth (linear) background with a sharp peak at a random location:

In [ ]:
import functools
from random import random

offset = random() - 0.5

def f(x, offset=0, wait=True):
    from time import sleep
    from random import random

    a = 0.01
    if wait:
        sleep(random())
    return x + a**2 / (a**2 + (x - offset)**2)

We start by initializing a 1D "learner", which will suggest points to evaluate, and adapt its suggestions as more and more points are evaluated.

In [ ]:
learner = adaptive.learner.Learner1D(f, bounds=(-1.0, 1.0))

Next we create a "runner" that will request points from the learner and evaluate 'func' on them.

By default the runner will evaluate the points in parallel using local processes ([`concurrent.futures.ProcessPoolExecutor`](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor)).

In [ ]:
# The end condition is when the "loss" is less than 0.1. In the context of the
# 1D learner this means that we will resolve features in 'func' with width 0.1 or wider.
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.1)

When instantiated in a Jupyter notebook the runner does its job in the background and does not block the IPython kernel.
We can use this to create a plot that updates as new data arrives:

In [ ]:
adaptive.live_plot(runner)

We can now compare the adaptive sampling to a homogeneous sampling with the same number of points:

In [ ]:
if not runner.task.done():
    raise RuntimeError('Wait for the runner to finish before executing the cells below!')

In [ ]:
import numpy as np

learner2 = adaptive.learner.Learner1D(f, bounds=(-1.01, 1.0))

xs = np.linspace(-1.0, 1.0, len(learner.data))
learner2.add_data(xs, map(functools.partial(func, wait=False), xs))

learner2.plot()

# Averaging learner

The next type of learner averages a function until the uncertainty in the average meets some condition.

This is useful for sampling a random variable. The function passed to the learner must formally take a single parameter,
which should be used like a "seed" for the (pseudo-) random variable (although in the current implementation the seed parameter can be ignored by the function).

In [ ]:
def g(n):
    import random
    from time import sleep
    sleep(random.random() / 5)
    # Properly save and restore the RNG state
    state = random.getstate()
    random.seed(n)
    val = random.gauss(0.5, 1)
    random.setstate(state)
    return val

In [ ]:
learner = adaptive.AverageLearner(g, None, 0.03)
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 1)
adaptive.live_plot(runner)

## Alternative executors

Often you will want to evaluate the function on some remote computing resources. `adaptive` works out of the box with any framework that implements a [PEP 3148](https://www.python.org/dev/peps/pep-3148/) compliant executor that returns `concurrent.futures.Future` objects.

### `concurrent.futures`

By default a runner creates a `ProcessPoolExecutor`, but you can also pass one explicitly e.g. to limit the number of workers:

In [ ]:
from concurrent.futures import ProcessPoolExecutor

executor = ProcessPoolExecutor(max_workers=4)

learner = adaptive.learner.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=executor, goal=lambda l: l.loss() < 0.1)
adaptive.live_plot(runner)

### IPyparallel

In [ ]:
import ipyparallel

client = ipyparallel.Client()
# f is a closure, so we have to use cloudpickle -- this is independent of 'adaptive'
client[:].use_cloudpickle()

learner = adaptive.learner.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=client, goal=lambda l: l.loss() < 0.1)
adaptive.live_plot(runner)

---

# Advanced Topics

## Cancelling a runner

Sometimes you want to interactively explore a parameter space, and want the function to be evaluated at finer and finer resolution and manually control when the calculation stops.

If no `goal` is provided to a runner then the runner will run until cancelled:

In [ ]:
learner = adaptive.learner.Learner1D(f, bounds=(-1.0, 1.0))
runner = adaptive.Runner(learner)
adaptive.live_plot(runner)

In [ ]:
runner.task.cancel()

## Debugging Problems 

Runners work in the background with respect to the IPython kernel, which makes it convenient, but also means that inspecting errors is more difficult because exceptions will not be raised directly in the notebook. Often the only indication you will have that something has gone wrong is that nothing will be happening.

Let's look at the following example, where the function to be learned will raise an exception 10% of the time.

In [ ]:
def will_raise(x):
    from random import random
    from time import sleep
    
    sleep(random())
    if random() < 0.1:
        raise RuntimeError('something went wrong!')
    return x**2
    
learner = adaptive.Learner1D(will_raise, (-1, 1))
runner = adaptive.Runner(learner)  # without 'goal' the runner will run forever unless cancelled
adaptive.live_plot(runner)

The above runner should continue forever, but we notice that it stops after a few points are evaluated.

First we should check that the runner has really finished:

In [ ]:
runner.task.done()

If it has indeed finished then we should check the `result` of the runner. This should be `None` if the runner stopped successfully. If the runner stopped due to an exception then asking for the result will raise the exception with the stack trace:

In [ ]:
runner.task.result()

## Using Runners from a script 

Runners can also be used from a Python script independently of the notebook:

```python
import adaptive

def f(x):
    return x

learner = adaptive.Learner1D(f, (-1, 1))

runner = adaptive.Runner(learner, goal=lambda: l: l.loss() < 0.1)
runner.run_sync()  # Block until completion.
```

Under the hood the runner uses [`asyncio`](https://docs.python.org/3/library/asyncio.html). You don't need to worry about this most of the time, unless your script uses asyncio itself. If this is the case you should be aware that instantiating a `Runner` schedules a new task on the current event loop, and you can simply

```python
    await runner.task
```

inside a coroutine to await completion of the runner.